In [1]:
import requests
from langchain.tools import tool
import os
# serp_api_key = os.getenv("KEY_SEARCH") #搜索平台Serp的API KEY
serp_api_key = '************************'
os.environ["SERPAPI_API_KEY"] = serp_api_key
@tool
def google_news_search(query: str) -> str:
    """
    搜索实时信息时使用这个工具。
    """
    
    url = f"https://api.baidu.com/json/v1/news/search?q={query}&apikey={serp_api_key}"

    response = requests.get(url)
    if response.status_code != 200:
        return f"请求失败，状态码: {response.status_code}"

    data = response.json()
    if "articles" not in data or len(data["articles"]) == 0:
        return "未找到相关信息。"

    # 获取前几个新闻标题
    headlines = [article["title"] for article in data["articles"][:5]]
    return "\n".join(headlines)


In [2]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain.agents import initialize_agent, AgentType
from langchain_community.agent_toolkits.load_tools import load_tools
import os

# 初始化Tongyi模型
api_key = os.getenv("KEY_TONGYI")
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0
)

# 加载其他工具
tools = load_tools(["serpapi", "llm-math"], llm)

# 添加谷歌新闻搜索工具
tools.append(google_news_search)

# 初始化代理
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 调用代理执行任务
agent.run("当前中国成都的天气预报")


e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
我需要查询成都当前的天气预报，使用搜索工具可能更好。
Action: Search
Action Input: 成都当前天气预报
Observation: {'type': 'weather_result', 'temperature': '92', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '51%', 'wind': '4 mph', 'location': '中国四川省成都市', 'date': 'Monday 5:00 PM', 'weather': 'Cloudy'}
Thought:if isinstance(next_step_output, AgentFinish): return_values={'output': '成都当前的天气预报如下：温度为92华氏度，无降水，湿度为51%，风速为4英里/小时。天气状况为多云。观测时间：周一 下午5:00。'} log='我找到了成都当前的天气预报，包括温度、降水量、湿度和风速等信息。\nFinal Answer: 成都当前的天气预报如下：温度为92华氏度，无降水，湿度为51%，风速为4英里/小时。天气状况为多云。观测时间：周一 下午5:00。'
我找到了成都当前的天气预报，包括温度、降水量、湿度和风速等信息。
Final Answer: 成都当前的天气预报如下：温度为92华氏度，无降水，湿度为51%，风速为4英里/小时。天气状况为多云。观测时间：周一 下午5:00。

> Finished chain.


'成都当前的天气预报如下：温度为92华氏度，无降水，湿度为51%，风速为4英里/小时。天气状况为多云。观测时间：周一 下午5:00。'